In [20]:
diabetes = pd.read_csv('diabetes.csv')
diabetes

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,measure
0,59.0,2.0,32.1,101.00,157.0,93.2,38.0,4.00,4.8598,87.0,151.0
1,48.0,1.0,21.6,87.00,183.0,103.2,70.0,3.00,3.8918,69.0,75.0
2,72.0,2.0,30.5,93.00,156.0,93.6,41.0,4.00,4.6728,85.0,141.0
3,24.0,1.0,25.3,84.00,198.0,131.4,40.0,5.00,4.8903,89.0,206.0
4,50.0,1.0,23.0,101.00,192.0,125.4,52.0,4.00,4.2905,80.0,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,60.0,2.0,28.2,112.00,185.0,113.8,42.0,4.00,4.9836,93.0,178.0
438,47.0,2.0,24.9,75.00,225.0,166.0,42.0,5.00,4.4427,102.0,104.0
439,60.0,2.0,24.9,99.67,162.0,106.6,43.0,3.77,4.1271,95.0,132.0
440,36.0,1.0,30.0,95.00,201.0,125.2,42.0,4.79,5.1299,85.0,220.0


In [22]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   age      442 non-null    float64
 1   sex      442 non-null    float64
 2   bmi      442 non-null    float64
 3   bp       442 non-null    float64
 4   s1       442 non-null    float64
 5   s2       442 non-null    float64
 6   s3       442 non-null    float64
 7   s4       442 non-null    float64
 8   s5       442 non-null    float64
 9   s6       442 non-null    float64
 10  measure  442 non-null    float64
dtypes: float64(11)
memory usage: 38.1 KB


In [23]:
diabetes.describe()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,measure
count,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,48.518100,1.468326,26.375792,94.647014,189.140271,115.439140,49.788462,4.070249,4.641411,91.260181,152.133484
std,13.109028,0.499561,4.418122,13.831283,34.608052,30.413081,12.934202,1.290450,0.522391,11.496335,77.093005
min,19.000000,1.000000,18.000000,62.000000,97.000000,41.600000,22.000000,2.000000,3.258100,58.000000,25.000000
25%,38.250000,1.000000,23.200000,84.000000,164.250000,96.050000,40.250000,3.000000,4.276700,83.250000,87.000000
50%,50.000000,1.000000,25.700000,93.000000,186.000000,113.000000,48.000000,4.000000,4.620050,91.000000,140.500000
75%,59.000000,2.000000,29.275000,105.000000,209.750000,134.500000,57.750000,5.000000,4.997200,98.000000,211.500000
max,79.000000,2.000000,42.200000,133.000000,301.000000,242.400000,99.000000,9.090000,6.107000,124.000000,346.000000


In [28]:
X = diabetes.drop(columns = 'measure')
y = diabetes['measure']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(353, 10) (89, 10) (353,) (89,)


In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [33]:
from sklearn.linear_model import RidgeCV

ridge = RidgeCV(alphas = np.logspace(-3, 1, 5))
ridge.fit(X_train_scaled, y_train)

ridge_y_pred = ridge.predict(X_test_scaled)

In [35]:
from sklearn.linear_model import LassoCV

lasso = LassoCV(alphas = np.logspace(-3, 1, 5))
lasso.fit(X_train_scaled, y_train)

lasso_y_pred = lasso.predict(X_test_scaled)

In [36]:
from sklearn.linear_model import ElasticNetCV

elastic = ElasticNetCV(alphas = np.logspace(-3, 1, 5))
elastic.fit(X_train_scaled, y_train)

elastic_y_pred = elastic.predict(X_test_scaled)

In [37]:
def adjusted_r2(r2, n, k):
    return 1 - (1 - r2) * (n - 1) / (n - k - 1)


from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Ridge
ridge_mse = mean_squared_error(y_test, ridge_y_pred)
ridge_r2 = r2_score(y_test, ridge_y_pred)
ridge_ad_r2 = adjusted_r2(ridge_r2, X_train.shape[0], X_train.shape[1])

# Lasso
lasso_mse = mean_squared_error(y_test, lasso_y_pred)
lasso_r2 = r2_score(y_test, lasso_y_pred)
lasso_ad_r2 = adjusted_r2(lasso_r2, X_train.shape[0], X_train.shape[1])

# ElasticNet
elastic_mse = mean_squared_error(y_test, elastic_y_pred)
elastic_r2 = r2_score(y_test, elastic_y_pred)
elastic_ad_r2 = adjusted_r2(elastic_r2, X_train.shape[0], X_train.shape[1])

print(
    "RMSE - Ridge: {0:0.4f}, Lasso: {1:0.4f}, ElasticNet: {2:0.4f}"
    .format(np.sqrt(ridge_mse), np.sqrt(lasso_mse), np.sqrt(elastic_mse))
)

print(
    "R2 - Ridge: {0:0.4f}, Lasso: {1:0.4f}, ElasticNet: {2:0.4f}"
    .format(ridge_r2, lasso_r2, elastic_r2)
)

print(
    "AD_R2 - Ridge: {0:0.4f}, Lasso: {1:0.4f}, ElasticNet: {2:0.4f}"
    .format(ridge_ad_r2, lasso_ad_r2, elastic_ad_r2)
)


RMSE - Ridge: 58.5597, Lasso: 58.4180, ElasticNet: 58.4211
R2 - Ridge: 0.3313, Lasso: 0.3345, ElasticNet: 0.3344
AD_R2 - Ridge: 0.3117, Lasso: 0.3150, ElasticNet: 0.3150


In [ ]:
# 연습문제
import numpy as np
import mglearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

# 1. 데이터 로드
X, y = mglearn.datasets.load_extended_boston()

# 2. Train : Test = 8 : 2 분할 (random_state = 0)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

# 3. MinMaxScaler로 스케일링
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. alpha 값 후보 설정
alphas = np.linspace(0.001, 10, 100)

best_alpha = None
best_score = -np.inf
best_model = None

# 5. alpha 탐색
for alpha in alphas:
    model = Ridge(alpha=alpha)
    model.fit(X_train_scaled, y_train)
    score = model.score(X_test_scaled, y_test)
    
    if score > best_score:
        best_score = score
        best_alpha = alpha
        best_model = model

# 6. 결과 출력
print("최적 alpha 값:", best_alpha)
print("테스트 세트 R^2 점수:", best_score)
print("절편 (intercept):")
print(best_model.intercept_)
print("회귀 계수 (coefficients):")
print(best_model.coef_)
